In [ ]:
# Thailand Map

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import tempfile
import warnings
warnings.filterwarnings("ignore")

# Function to get a writable directory
def get_writable_dir():
    """Returns a directory path where we can write files"""
    # Try user's home directory
    home_dir = Path.home()
    if os.access(home_dir, os.W_OK):
        return home_dir
    
    # If home directory isn't writable, use system temp directory
    return Path(tempfile.gettempdir())

# Get a writable directory for saving our images
save_dir = get_writable_dir()
print(f"Saving maps to: {save_dir}")

# Define village data with updated precise coordinates
village_data = [
    {"Village": "Huay So", "Latitude": 20.04725130852227, "Longitude": 100.29453349604997, 
     "District": "Chiang Khong District"},
    {"Village": "Ngam Mueang", "Latitude": 19.92264589955456, "Longitude": 100.31612106483033, 
     "District": "Khun Tan District"},
    {"Village": "Bun Rueang Tai", "Latitude": 19.994183778980528, "Longitude": 100.33864125722558, 
     "District": "Chiang Khong District"},
    {"Village": "Huay Sak", "Latitude": 19.884345802551717, "Longitude": 100.27573371862452, 
     "District": "Khun Tan District"},
    {"Village": "Muang Chum", "Latitude": 20.1023100075089, "Longitude": 100.41152965899933, 
     "District": "Chiang Khong District"}
]

# Create DataFrame with village data
village_df = pd.DataFrame(village_data)

print("Village Data with Updated Coordinates:")
print(village_df[['Village', 'Latitude', 'Longitude', 'District']])

# Check if contextily is installed, otherwise try to install it
try:
    import contextily as ctx
    print("Contextily is already installed and imported.")
except ImportError:
    print("Contextily not found. Attempting to install...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "contextily"])
    try:
        import contextily as ctx
        print("Contextily successfully installed and imported.")
    except ImportError:
        print("Failed to install contextily. Will create a simple map instead.")
        ctx = None

# Function to create detailed map with actual geographic data
def create_actual_map():
    if 'ctx' not in globals():
        print("Contextily not available. Creating a simple map instead.")
        create_simple_map_fallback()
        return

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 10))  # Size large enough for Thailand

    # Thailand boundaries
    ax.set_xlim(97.0, 106.0)
    ax.set_ylim(5.5, 21.0)
    ax.set_aspect('equal')  # Adjust proportion appropriately

    # Try different basemap types
    try:
        basemap_sources = [
            ('OpenStreetMap',
             lambda ax: ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=6, crs='EPSG:4326')),
            ('ESRI World Topo',
             lambda ax: ctx.add_basemap(ax, source=ctx.providers.Esri.WorldTopoMap, zoom=6, crs='EPSG:4326')),
            ('Stamen Terrain',
             lambda ax: ctx.add_basemap(ax, source=ctx.providers.Stamen.Terrain, zoom=6, crs='EPSG:4326'))
        ]

        basemap_added = False
        for name, add_basemap_func in basemap_sources:
            try:
                add_basemap_func(ax)
                print(f"Added {name} basemap successfully")
                basemap_added = True
                break
            except Exception as e:
                print(f"Could not add {name} basemap: {e}")

        if not basemap_added:
            print("Could not add any basemap.")
            plt.close(fig)
            return

    except Exception as e:
        print(f"Error adding basemap: {e}")
        plt.close(fig)
        return

    # Map title
    plt.title('Map of Thailand with Chiang Rai Marker', fontsize=14, pad=12)

    # Add marker for Chiang Rai province
    chiang_rai_lat = 19.90
    chiang_rai_lon = 100.10

    ax.scatter(
        chiang_rai_lon, chiang_rai_lat,
        c='red',
        s=100,
        marker='o',
        edgecolors='black',
        linewidths=1.5,
        zorder=10,
        label='Chiang Rai'
    )

    # Add "Chiang Rai" label
    ax.annotate(
        'Chiang Rai',
        xy=(chiang_rai_lon, chiang_rai_lat),
        xytext=(0, 10),
        textcoords='offset points',
        ha='center',
        fontsize=9,
        fontweight='bold',
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='red', boxstyle='round,pad=0.3')
    )

    # Add North arrow
    arrow_x = 105.5
    arrow_y = 6.5
    arrow_len = 1.0

    ax.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
                arrowprops=dict(facecolor='black', width=2, headwidth=8), zorder=10)
    ax.text(arrow_x, arrow_y + arrow_len + 0.3, 'N', ha='center', va='center',
            fontsize=10, fontweight='bold',
            bbox=dict(facecolor='white', alpha=0.9, edgecolor='black', pad=2))

    # Add Scale bar
    scale_km = 200
    scale_deg = scale_km / 111

    scale_x = 97.5
    scale_y = 6.0

    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'w-', linewidth=4, zorder=20)
    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'k-', linewidth=2, zorder=21)
    ax.plot([scale_x, scale_x], [scale_y - 0.2, scale_y + 0.2], 'k-', linewidth=1.5)
    ax.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y - 0.2, scale_y + 0.2], 'k-', linewidth=1.5)
    ax.text(scale_x + scale_deg / 2, scale_y - 0.4, f'{scale_km} km', ha='center', fontsize=8,
            bbox=dict(facecolor='white', alpha=0.9, edgecolor='black', pad=2), zorder=21)

    # Attribution
    plt.figtext(0.01, 0.01, "Basemap: © OpenStreetMap contributors", fontsize=6, style='italic')

    # Save map
    save_path = save_dir / "thailand_map_with_chiang_rai.png"
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        print(f"Map with Chiang Rai saved to: {save_path}")
    except Exception as e:
        print(f"Error saving map: {e}")

    plt.show()


# Fallback function if contextily is not available
def create_simple_map_fallback():
    from matplotlib.patches import Polygon
    from matplotlib.lines import Line2D
    
    # Create figure with standard size
    plt.figure(figsize=(10, 8))
    
    # Calculate center of villages
    center_lon = village_df['Longitude'].mean()
    center_lat = village_df['Latitude'].mean()
    
    # Calculate range of coordinates
    lon_range = village_df['Longitude'].max() - village_df['Longitude'].min()
    lat_range = village_df['Latitude'].max() - village_df['Latitude'].min()
    
    # Use moderate padding for closer zoom
    padding_factor = max(lon_range, lat_range) * 1.2
    
    # Calculate bounds
    min_lon = center_lon - padding_factor
    max_lon = center_lon + padding_factor
    min_lat = center_lat - padding_factor
    max_lat = center_lat + padding_factor
    
    # Set axis limits
    plt.xlim(min_lon, max_lon)
    plt.ylim(min_lat, max_lat)
    
    # Add a light background color to simulate a map
    plt.gca().set_facecolor('#f2f2f2')
    
    # Create district colors
    district_colors = {
        'Chiang Khong District': 'blue',
        'Khun Tan District': 'green'
    }
    
    # Create district markers
    district_markers = {
        'Chiang Khong District': 'o',  # circle
        'Khun Tan District': 's'       # square
    }
    
    # Plot villages by district
    for district in district_colors:
        district_df = village_df[village_df['District'] == district]
        
        plt.scatter(
            district_df['Longitude'], 
            district_df['Latitude'],
            c=district_colors[district],
            marker=district_markers[district],
            s=120,
            alpha=0.8,
            edgecolors='black',
            linewidths=1.5,
            zorder=10,
            label=district
        )
    
    # Add village labels (no subdistricts)
    for i, row in village_df.iterrows():
        district_color = district_colors.get(row['District'], 'black')
        
        plt.annotate(
            row['Village'], 
            xy=(row['Longitude'], row['Latitude']),
            xytext=(0, 10),
            textcoords='offset points',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold',
            bbox=dict(
                boxstyle="round,pad=0.3",
                fc="white",
                ec=district_color,
                linewidth=1.5,
                alpha=0.9
            ),
            zorder=11
        )
    
    # Create approximate district boundaries
    # Chiang Khong District
    chiang_khong = village_df[village_df['District'] == 'Chiang Khong District']
    if len(chiang_khong) > 1:
        ck_min_lon = chiang_khong['Longitude'].min() - 0.02
        ck_max_lon = chiang_khong['Longitude'].max() + 0.02
        ck_min_lat = chiang_khong['Latitude'].min() - 0.02
        ck_max_lat = chiang_khong['Latitude'].max() + 0.02
        
        district_box = Polygon([
            [ck_min_lon, ck_min_lat],
            [ck_min_lon, ck_max_lat],
            [ck_max_lon, ck_max_lat],
            [ck_max_lon, ck_min_lat]
        ], closed=True, fill=True, edgecolor='blue', facecolor='blue', alpha=0.1, zorder=5)
        plt.gca().add_patch(district_box)
    
    # Khun Tan District
    khun_tan = village_df[village_df['District'] == 'Khun Tan District']
    if len(khun_tan) > 1:
        kt_min_lon = khun_tan['Longitude'].min() - 0.02
        kt_max_lon = khun_tan['Longitude'].max() + 0.02
        kt_min_lat = khun_tan['Latitude'].min() - 0.02
        kt_max_lat = khun_tan['Latitude'].max() + 0.02
        
        district_box = Polygon([
            [kt_min_lon, kt_min_lat],
            [kt_min_lon, kt_max_lat],
            [kt_max_lon, kt_max_lat],
            [kt_max_lon, kt_min_lat]
        ], closed=True, fill=True, edgecolor='green', facecolor='green', alpha=0.1, zorder=5)
        plt.gca().add_patch(district_box)
    
    # Add scale bar
    scale_km = 5  # km
    scale_deg = scale_km / 111  # approximate at equator
    
    scale_x = min_lon + (max_lon - min_lon) * 0.05
    scale_y = min_lat + (max_lat - min_lat) * 0.05
    
    plt.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'k-', linewidth=2, zorder=15)
    plt.plot([scale_x, scale_x], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=15)
    plt.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=15)
    plt.text(scale_x + scale_deg/2, scale_y - 0.01, f'{scale_km} km', ha='center', fontsize=8, zorder=15)
    
    # Add north arrow
    arrow_x = max_lon - (max_lon - min_lon) * 0.05
    arrow_y = min_lat + (max_lat - min_lat) * 0.05
    arrow_len = (max_lat - min_lat) * 0.03
    
    plt.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
                arrowprops=dict(facecolor='black', width=1.5, headwidth=6), zorder=15)
    plt.text(arrow_x, arrow_y + arrow_len + 0.005, 'N', ha='center', va='center', 
             fontsize=9, fontweight='bold', zorder=15)
    
    # Add grid and labels
    plt.grid(True, linestyle='--', alpha=0.4, color='gray', zorder=0)
    plt.xlabel('Longitude', fontsize=9)
    plt.ylabel('Latitude', fontsize=9)
    
    # Add legend
    custom_lines = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markeredgecolor='black', 
               markersize=6, label='Villages in Chiang Khong District'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='green', markeredgecolor='black', 
               markersize=6, label='Villages in Khun Tan District')
    ]
    plt.legend(
        handles=custom_lines, 
        loc='upper right', 
        fontsize=8,
        title="Villages by District",
        title_fontsize=9
    )
    
    # Add title
    plt.title('Villages in Chiang Rai Province, Thailand', fontsize=12)
    
    # Save the map
    save_path = save_dir / "chiang_rai_villages_map_final_simple.png"
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        print(f"Simple map saved to: {save_path}")
    except Exception as e:
        print(f"Error saving map: {e}")
    
    plt.show()

# Create the map
print("\nCreating final map...")
create_actual_map()

print("\nMap generation complete!")

In [ ]:
# Chiang Rai Map

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import tempfile
import warnings
warnings.filterwarnings("ignore")

# Function to get a writable directory
def get_writable_dir():
    """Returns a directory path where we can write files"""
    # Try user's home directory
    home_dir = Path.home()
    if os.access(home_dir, os.W_OK):
        return home_dir
    
    # If home directory isn't writable, use system temp directory
    return Path(tempfile.gettempdir())

# Get a writable directory for saving our images
save_dir = get_writable_dir()
print(f"Saving maps to: {save_dir}")

# Define village data with updated precise coordinates
village_data = [
    {"Village": "Huay So", "Latitude": 20.04725130852227, "Longitude": 100.29453349604997, 
     "District": "Chiang Khong District"},
    {"Village": "Ngam Mueang", "Latitude": 19.92264589955456, "Longitude": 100.31612106483033, 
     "District": "Khun Tan District"},
    {"Village": "Bun Rueang Tai", "Latitude": 19.994183778980528, "Longitude": 100.33864125722558, 
     "District": "Chiang Khong District"},
    {"Village": "Huay Sak", "Latitude": 19.884345802551717, "Longitude": 100.27573371862452, 
     "District": "Khun Tan District"},
    {"Village": "Muang Chum", "Latitude": 20.1023100075089, "Longitude": 100.41152965899933, 
     "District": "Chiang Khong District"}
]

# Create DataFrame with village data
village_df = pd.DataFrame(village_data)

print("Village Data with Updated Coordinates:")
print(village_df[['Village', 'Latitude', 'Longitude', 'District']])

# Check if contextily is installed, otherwise try to install it
try:
    import contextily as ctx
    print("Contextily is already installed and imported.")
except ImportError:
    print("Contextily not found. Attempting to install...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "contextily"])
    try:
        import contextily as ctx
        print("Contextily successfully installed and imported.")
    except ImportError:
        print("Failed to install contextily. Will create a simple map instead.")
        ctx = None

# Function to create detailed map with actual geographic data
def create_actual_map():
    if 'ctx' not in globals():
        print("Contextily not available. Creating a simple map instead.")
        create_simple_map_fallback()
        return

    fig, ax = plt.subplots(figsize=(10, 9))

    # ✅ Zoom to central Chiang Rai
    ax.set_xlim(99.5, 101.3)
    ax.set_ylim(19.3, 20.6)
    ax.set_aspect('auto')

    try:
        ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=10, crs='EPSG:4326')
        print("Basemap added successfully")
    except Exception as e:
        print(f"Error adding basemap: {e}")
        plt.close(fig)
        return

    # 🔴 Highlight point: Chiang Rai Province
    chiang_rai_lat = 19.90
    chiang_rai_lon = 100.10

    ax.scatter(chiang_rai_lon, chiang_rai_lat,
               c='red', s=120, marker='o',
               edgecolors='black', linewidths=1.5,
               zorder=10)

    ax.annotate('Chiang Rai',
                xy=(chiang_rai_lon, chiang_rai_lat),
                xytext=(0, 10),
                textcoords='offset points',
                ha='center',
                fontsize=10, fontweight='bold',
                bbox=dict(facecolor='white', edgecolor='red',
                          boxstyle='round,pad=0.3', alpha=0.85),
                zorder=11)

    # North arrow in the bottom right
    arrow_x = 101.1
    arrow_y = 19.35
    arrow_len = 0.1

    ax.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
                arrowprops=dict(facecolor='black', width=2, headwidth=6), zorder=12)
    ax.text(arrow_x, arrow_y + arrow_len + 0.02, 'N',
            ha='center', fontsize=9, fontweight='bold',
            bbox=dict(facecolor='white', edgecolor='black', pad=2, alpha=0.9))

    # Scale bar in the bottom left
    scale_km = 10
    scale_deg = scale_km / 111
    scale_x = 99.6
    scale_y = 19.35

    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'w-', linewidth=4, zorder=20)
    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'k-', linewidth=2, zorder=21)
    ax.plot([scale_x, scale_x], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5)
    ax.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5)
    ax.text(scale_x + scale_deg / 2, scale_y - 0.015, f'{scale_km} km',
            ha='center', fontsize=8,
            bbox=dict(facecolor='white', alpha=0.9, edgecolor='black', pad=2), zorder=21)

    # Map title
    plt.title('Chiang Rai Province (Zoomed In)', fontsize=13, pad=10)

    # Attribution
    plt.figtext(0.01, 0.01, "Basemap: © OpenStreetMap contributors", fontsize=6, style='italic')

    # Save
    save_path = save_dir / "chiang_rai_zoomed_map.png"
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        print(f"Zoomed-in Chiang Rai map saved to: {save_path}")
    except Exception as e:
        print(f"Error saving map: {e}")

    plt.show()




# Fallback function if contextily is not available
def create_simple_map_fallback():
    from matplotlib.patches import Polygon
    from matplotlib.lines import Line2D
    
    # Create figure with standard size
    plt.figure(figsize=(10, 8))
    
    # Calculate center of villages
    center_lon = village_df['Longitude'].mean()
    center_lat = village_df['Latitude'].mean()
    
    # Calculate range of coordinates
    lon_range = village_df['Longitude'].max() - village_df['Longitude'].min()
    lat_range = village_df['Latitude'].max() - village_df['Latitude'].min()
    
    # Use moderate padding for closer zoom
    padding_factor = max(lon_range, lat_range) * 1.2
    
    # Calculate bounds
    min_lon = center_lon - padding_factor
    max_lon = center_lon + padding_factor
    min_lat = center_lat - padding_factor
    max_lat = center_lat + padding_factor
    
    # Set axis limits
    plt.xlim(min_lon, max_lon)
    plt.ylim(min_lat, max_lat)
    
    # Add a light background color to simulate a map
    plt.gca().set_facecolor('#f2f2f2')
    
    # Create district colors
    district_colors = {
        'Chiang Khong District': 'blue',
        'Khun Tan District': 'green'
    }
    
    # Create district markers
    district_markers = {
        'Chiang Khong District': 'o',  # circle
        'Khun Tan District': 's'       # square
    }
    
    # Plot villages by district
    for district in district_colors:
        district_df = village_df[village_df['District'] == district]
        
        plt.scatter(
            district_df['Longitude'], 
            district_df['Latitude'],
            c=district_colors[district],
            marker=district_markers[district],
            s=120,
            alpha=0.8,
            edgecolors='black',
            linewidths=1.5,
            zorder=10,
            label=district
        )
    
    # Add village labels (no subdistricts)
    for i, row in village_df.iterrows():
        district_color = district_colors.get(row['District'], 'black')
        
        plt.annotate(
            row['Village'], 
            xy=(row['Longitude'], row['Latitude']),
            xytext=(0, 10),
            textcoords='offset points',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold',
            bbox=dict(
                boxstyle="round,pad=0.3",
                fc="white",
                ec=district_color,
                linewidth=1.5,
                alpha=0.9
            ),
            zorder=11
        )
    
    # Create approximate district boundaries
    # Chiang Khong District
    chiang_khong = village_df[village_df['District'] == 'Chiang Khong District']
    if len(chiang_khong) > 1:
        ck_min_lon = chiang_khong['Longitude'].min() - 0.02
        ck_max_lon = chiang_khong['Longitude'].max() + 0.02
        ck_min_lat = chiang_khong['Latitude'].min() - 0.02
        ck_max_lat = chiang_khong['Latitude'].max() + 0.02
        
        district_box = Polygon([
            [ck_min_lon, ck_min_lat],
            [ck_min_lon, ck_max_lat],
            [ck_max_lon, ck_max_lat],
            [ck_max_lon, ck_min_lat]
        ], closed=True, fill=True, edgecolor='blue', facecolor='blue', alpha=0.1, zorder=5)
        plt.gca().add_patch(district_box)
    
    # Khun Tan District
    khun_tan = village_df[village_df['District'] == 'Khun Tan District']
    if len(khun_tan) > 1:
        kt_min_lon = khun_tan['Longitude'].min() - 0.02
        kt_max_lon = khun_tan['Longitude'].max() + 0.02
        kt_min_lat = khun_tan['Latitude'].min() - 0.02
        kt_max_lat = khun_tan['Latitude'].max() + 0.02
        
        district_box = Polygon([
            [kt_min_lon, kt_min_lat],
            [kt_min_lon, kt_max_lat],
            [kt_max_lon, kt_max_lat],
            [kt_max_lon, kt_min_lat]
        ], closed=True, fill=True, edgecolor='green', facecolor='green', alpha=0.1, zorder=5)
        plt.gca().add_patch(district_box)
    
    # Add scale bar
    scale_km = 5  # km
    scale_deg = scale_km / 111  # approximate at equator
    
    scale_x = min_lon + (max_lon - min_lon) * 0.05
    scale_y = min_lat + (max_lat - min_lat) * 0.05
    
    plt.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'k-', linewidth=2, zorder=15)
    plt.plot([scale_x, scale_x], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=15)
    plt.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=15)
    plt.text(scale_x + scale_deg/2, scale_y - 0.01, f'{scale_km} km', ha='center', fontsize=8, zorder=15)
    
    # Add north arrow
    arrow_x = max_lon - (max_lon - min_lon) * 0.05
    arrow_y = min_lat + (max_lat - min_lat) * 0.05
    arrow_len = (max_lat - min_lat) * 0.03
    
    plt.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
                arrowprops=dict(facecolor='black', width=1.5, headwidth=6), zorder=15)
    plt.text(arrow_x, arrow_y + arrow_len + 0.005, 'N', ha='center', va='center', 
             fontsize=9, fontweight='bold', zorder=15)
    
    # Add grid and labels
    plt.grid(True, linestyle='--', alpha=0.4, color='gray', zorder=0)
    plt.xlabel('Longitude', fontsize=9)
    plt.ylabel('Latitude', fontsize=9)
    
    # Add legend
    custom_lines = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markeredgecolor='black', 
               markersize=6, label='Villages in Chiang Khong District'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='green', markeredgecolor='black', 
               markersize=6, label='Villages in Khun Tan District')
    ]
    plt.legend(
        handles=custom_lines, 
        loc='upper right', 
        fontsize=8,
        title="Villages by District",
        title_fontsize=9
    )
    
    # Add title
    plt.title('Villages in Chiang Rai Province, Thailand', fontsize=12)
    
    # Save the map
    save_path = save_dir / "chiang_rai_villages_map_final_simple.png"
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        print(f"Simple map saved to: {save_path}")
    except Exception as e:
        print(f"Error saving map: {e}")
    
    plt.show()

# Create the map
print("\nCreating final map...")
create_actual_map()

print("\nMap generation complete!")

In [ ]:
# Villages Map

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import tempfile
import warnings
warnings.filterwarnings("ignore")

# Function to get a writable directory
def get_writable_dir():
    """Returns a directory path where we can write files"""
    # Try user's home directory
    home_dir = Path.home()
    if os.access(home_dir, os.W_OK):
        return home_dir
    
    # If home directory isn't writable, use system temp directory
    return Path(tempfile.gettempdir())

# Get a writable directory for saving our images
save_dir = get_writable_dir()
print(f"Saving maps to: {save_dir}")

# Define village data with updated precise coordinates
village_data = [
    {"Village": "Huay So", "Latitude": 20.04725130852227, "Longitude": 100.29453349604997, 
     "District": "Chiang Khong District"},
    {"Village": "Ngam Mueang", "Latitude": 19.92264589955456, "Longitude": 100.31612106483033, 
     "District": "Khun Tan District"},
    {"Village": "Bun Rueang Tai", "Latitude": 19.994183778980528, "Longitude": 100.33864125722558, 
     "District": "Chiang Khong District"},
    {"Village": "Huay Sak", "Latitude": 19.884345802551717, "Longitude": 100.27573371862452, 
     "District": "Khun Tan District"},
    {"Village": "Muang Chum", "Latitude": 20.1023100075089, "Longitude": 100.41152965899933, 
     "District": "Chiang Khong District"}
]

# Create DataFrame with village data
village_df = pd.DataFrame(village_data)

print("Village Data with Updated Coordinates:")
print(village_df[['Village', 'Latitude', 'Longitude', 'District']])

# Check if contextily is installed, otherwise try to install it
try:
    import contextily as ctx
    print("Contextily is already installed and imported.")
except ImportError:
    print("Contextily not found. Attempting to install...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "contextily"])
    try:
        import contextily as ctx
        print("Contextily successfully installed and imported.")
    except ImportError:
        print("Failed to install contextily. Will create a simple map instead.")
        ctx = None

# Function to create detailed map with actual geographic data
def create_actual_map():
    if 'ctx' not in globals():
        print("Contextily not available. Creating a simple map instead.")
        create_simple_map_fallback()
        return
    
    # Create figure and axis with standard size
    fig, ax = plt.subplots(figsize=(10, 8))  # Standard size, not expanded
    
    # Define district colors and markers
    district_colors = {
        'Chiang Khong District': 'blue',
        'Khun Tan District': 'green'
    }
    
    district_markers = {
        'Chiang Khong District': 'o',  # circle
        'Khun Tan District': 's'       # square
    }
    
    # Calculate the center of the villages
    center_lon = village_df['Longitude'].mean()
    center_lat = village_df['Latitude'].mean()
    
    # Calculate the range of the coordinates
    lon_range = village_df['Longitude'].max() - village_df['Longitude'].min()
    lat_range = village_df['Latitude'].max() - village_df['Latitude'].min()
    
    # Use less padding for closer zoom (1.2x instead of 2x)
    padding_factor = max(lon_range, lat_range) * 1.2
    
    # Calculate the new bounds
    min_lon = center_lon - padding_factor
    max_lon = center_lon + padding_factor
    min_lat = center_lat - padding_factor
    max_lat = center_lat + padding_factor
    
    # Set the limits
    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)
    
    # Add basemap
    try:
        # Try different basemap sources
        basemap_sources = [
            ('OpenStreetMap', 
             lambda ax: ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=10, crs='EPSG:4326')),
            ('ESRI World Topo', 
             lambda ax: ctx.add_basemap(ax, source=ctx.providers.Esri.WorldTopoMap, zoom=10, crs='EPSG:4326')),
            ('Stamen Terrain', 
             lambda ax: ctx.add_basemap(ax, source=ctx.providers.Stamen.Terrain, zoom=10, crs='EPSG:4326'))
        ]
        
        basemap_added = False
        for name, add_basemap_func in basemap_sources:
            try:
                add_basemap_func(ax)
                print(f"Added {name} basemap successfully")
                basemap_added = True
                break
            except Exception as e:
                print(f"Could not add {name} basemap: {e}")
        
        if not basemap_added:
            print("Could not add any basemap. Creating a simple map instead.")
            plt.close(fig)
            create_simple_map_fallback()
            return
        
    except Exception as e:
        print(f"Error adding basemap: {e}")
        print("Creating a simple map instead.")
        plt.close(fig)
        create_simple_map_fallback()
        return
    
    # Plot villages by district
    for district in village_df['District'].unique():
        district_villages = village_df[village_df['District'] == district]
        ax.scatter(
            district_villages['Longitude'],
            district_villages['Latitude'],
            c=district_colors.get(district, 'red'),
            marker=district_markers.get(district, 'o'),
            s=120,  # Marker size
            alpha=0.8,
            edgecolors='black',
            linewidths=1.5,
            label=f"Villages in {district}",
            zorder=10
        )
    
    # Add village labels (no subdistricts)
    for i, village in village_df.iterrows():
        # Village name with box
        ax.annotate(
            village['Village'],
            xy=(village['Longitude'], village['Latitude']),
            xytext=(0, 10),  # Offset above
            textcoords="offset points",
            fontsize=10,  # Smaller font size for standard map
            fontweight='bold',
            ha='center',
            va='bottom',
            bbox=dict(
                boxstyle="round,pad=0.3",
                fc="white",
                ec=district_colors.get(village['District'], 'red'),
                linewidth=1.5,
                alpha=0.9
            ),
            zorder=12
        )
    
    # Add title (only main title, no subtitle)
    plt.title('Villages in Chiang Rai Province, Thailand', fontsize=12, pad=10)
    
    # Add scale bar
    scale_km = 5  # km
    scale_deg = scale_km / 111  # approximate at equator
    
    scale_x = min_lon + (max_lon - min_lon) * 0.05
    scale_y = min_lat + (max_lat - min_lat) * 0.05
    
    # White background
    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'w-', linewidth=4, zorder=20)
    # Black line
    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'k-', linewidth=2, zorder=21)
    ax.plot([scale_x, scale_x], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=21)
    ax.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=21)
    
    # Add scale label with background
    ax.text(scale_x + scale_deg/2, scale_y - 0.01, f'{scale_km} km', ha='center', fontsize=8,
           bbox=dict(facecolor='white', alpha=0.9, edgecolor='black', pad=2), zorder=21)
    
    # Add north arrow
    arrow_x = max_lon - (max_lon - min_lon) * 0.05
    arrow_y = min_lat + (max_lat - min_lat) * 0.05
    arrow_len = (max_lat - min_lat) * 0.03
    
    # White background arrow
    ax.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
               arrowprops=dict(facecolor='white', edgecolor='black', width=3, headwidth=8), zorder=20)
    # Black arrow
    ax.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
               arrowprops=dict(facecolor='black', width=1.5, headwidth=6), zorder=21)
    
    # Add 'N' label
    ax.text(arrow_x, arrow_y + arrow_len + 0.005, 'N', ha='center', va='center', 
           fontsize=9, fontweight='bold',
           bbox=dict(facecolor='white', alpha=0.9, edgecolor='black', pad=2), zorder=21)
    
    # Update legend to be more compact
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markeredgecolor='black', 
               markersize=6, label='Villages in Chiang Khong District'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='green', markeredgecolor='black', 
               markersize=6, label='Villages in Khun Tan District')
    ]
    
    legend = ax.legend(
        handles=legend_elements,
        loc='upper right', 
        framealpha=0.9, 
        fontsize=8,
        title="Villages by District",
        title_fontsize=9
    )
    legend.set_zorder(20)
    
    # Add attribution for the basemap
    plt.figtext(0.01, 0.01, 
                "Basemap: © OpenStreetMap contributors",
                fontsize=6, style='italic')
    
    # Save the map with high resolution
    save_path = save_dir / "chiang_rai_villages_map_final.png"
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        print(f"Map saved to: {save_path}")
    except Exception as e:
        print(f"Error saving map: {e}")
    
    plt.show()

# Fallback function if contextily is not available
def create_simple_map_fallback():
    from matplotlib.patches import Polygon
    from matplotlib.lines import Line2D
    
    # Create figure with standard size
    plt.figure(figsize=(10, 8))
    
    # Calculate center of villages
    center_lon = village_df['Longitude'].mean()
    center_lat = village_df['Latitude'].mean()
    
    # Calculate range of coordinates
    lon_range = village_df['Longitude'].max() - village_df['Longitude'].min()
    lat_range = village_df['Latitude'].max() - village_df['Latitude'].min()
    
    # Use moderate padding for closer zoom
    padding_factor = max(lon_range, lat_range) * 1.2
    
    # Calculate bounds
    min_lon = center_lon - padding_factor
    max_lon = center_lon + padding_factor
    min_lat = center_lat - padding_factor
    max_lat = center_lat + padding_factor
    
    # Set axis limits
    plt.xlim(min_lon, max_lon)
    plt.ylim(min_lat, max_lat)
    
    # Add a light background color to simulate a map
    plt.gca().set_facecolor('#f2f2f2')
    
    # Create district colors
    district_colors = {
        'Chiang Khong District': 'blue',
        'Khun Tan District': 'green'
    }
    
    # Create district markers
    district_markers = {
        'Chiang Khong District': 'o',  # circle
        'Khun Tan District': 's'       # square
    }
    
    # Plot villages by district
    for district in district_colors:
        district_df = village_df[village_df['District'] == district]
        
        plt.scatter(
            district_df['Longitude'], 
            district_df['Latitude'],
            c=district_colors[district],
            marker=district_markers[district],
            s=120,
            alpha=0.8,
            edgecolors='black',
            linewidths=1.5,
            zorder=10,
            label=district
        )
    
    # Add village labels (no subdistricts)
    for i, row in village_df.iterrows():
        district_color = district_colors.get(row['District'], 'black')
        
        plt.annotate(
            row['Village'], 
            xy=(row['Longitude'], row['Latitude']),
            xytext=(0, 10),
            textcoords='offset points',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold',
            bbox=dict(
                boxstyle="round,pad=0.3",
                fc="white",
                ec=district_color,
                linewidth=1.5,
                alpha=0.9
            ),
            zorder=11
        )
    
    # Create approximate district boundaries
    # Chiang Khong District
    chiang_khong = village_df[village_df['District'] == 'Chiang Khong District']
    if len(chiang_khong) > 1:
        ck_min_lon = chiang_khong['Longitude'].min() - 0.02
        ck_max_lon = chiang_khong['Longitude'].max() + 0.02
        ck_min_lat = chiang_khong['Latitude'].min() - 0.02
        ck_max_lat = chiang_khong['Latitude'].max() + 0.02
        
        district_box = Polygon([
            [ck_min_lon, ck_min_lat],
            [ck_min_lon, ck_max_lat],
            [ck_max_lon, ck_max_lat],
            [ck_max_lon, ck_min_lat]
        ], closed=True, fill=True, edgecolor='blue', facecolor='blue', alpha=0.1, zorder=5)
        plt.gca().add_patch(district_box)
    
    # Khun Tan District
    khun_tan = village_df[village_df['District'] == 'Khun Tan District']
    if len(khun_tan) > 1:
        kt_min_lon = khun_tan['Longitude'].min() - 0.02
        kt_max_lon = khun_tan['Longitude'].max() + 0.02
        kt_min_lat = khun_tan['Latitude'].min() - 0.02
        kt_max_lat = khun_tan['Latitude'].max() + 0.02
        
        district_box = Polygon([
            [kt_min_lon, kt_min_lat],
            [kt_min_lon, kt_max_lat],
            [kt_max_lon, kt_max_lat],
            [kt_max_lon, kt_min_lat]
        ], closed=True, fill=True, edgecolor='green', facecolor='green', alpha=0.1, zorder=5)
        plt.gca().add_patch(district_box)
    
    # Add scale bar
    scale_km = 5  # km
    scale_deg = scale_km / 111  # approximate at equator
    
    scale_x = min_lon + (max_lon - min_lon) * 0.05
    scale_y = min_lat + (max_lat - min_lat) * 0.05
    
    plt.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 'k-', linewidth=2, zorder=15)
    plt.plot([scale_x, scale_x], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=15)
    plt.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y - 0.005, scale_y + 0.005], 'k-', linewidth=1.5, zorder=15)
    plt.text(scale_x + scale_deg/2, scale_y - 0.01, f'{scale_km} km', ha='center', fontsize=8, zorder=15)
    
    # Add north arrow
    arrow_x = max_lon - (max_lon - min_lon) * 0.05
    arrow_y = min_lat + (max_lat - min_lat) * 0.05
    arrow_len = (max_lat - min_lat) * 0.03
    
    plt.annotate('', xy=(arrow_x, arrow_y + arrow_len), xytext=(arrow_x, arrow_y),
                arrowprops=dict(facecolor='black', width=1.5, headwidth=6), zorder=15)
    plt.text(arrow_x, arrow_y + arrow_len + 0.005, 'N', ha='center', va='center', 
             fontsize=9, fontweight='bold', zorder=15)
    
    # Add grid and labels
    plt.grid(True, linestyle='--', alpha=0.4, color='gray', zorder=0)
    plt.xlabel('Longitude', fontsize=9)
    plt.ylabel('Latitude', fontsize=9)
    
    # Add legend
    custom_lines = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markeredgecolor='black', 
               markersize=6, label='Villages in Chiang Khong District'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='green', markeredgecolor='black', 
               markersize=6, label='Villages in Khun Tan District')
    ]
    plt.legend(
        handles=custom_lines, 
        loc='upper right', 
        fontsize=8,
        title="Villages by District",
        title_fontsize=9
    )
    
    # Add title
    plt.title('Villages in Chiang Rai Province, Thailand', fontsize=12)
    
    # Save the map
    save_path = save_dir / "chiang_rai_villages_map_final_simple.png"
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        print(f"Simple map saved to: {save_path}")
    except Exception as e:
        print(f"Error saving map: {e}")
    
    plt.show()

# Create the map
print("\nCreating final map...")
create_actual_map()

print("\nMap generation complete!")
